In [1]:
import scipy
import scipy.sparse, scipy.stats
import numpy
import os
import re
import multiprocessing
import subprocess
import pickle
import matplotlib.pyplot
import time
import statistics
import sklearn
import statsmodels , statsmodels.api
import itertools

In [2]:
def moving_integration (values, window):
    weights = numpy.repeat(1.0, window)
    sma = numpy.convolve(values, weights, 'same')
    return sma
def moving_average (values, window):
    weights = numpy.repeat(1.0, window)/window
    sma = numpy.convolve(values, weights, 'same')
    return sma

In [3]:
#needed to load the variables
with open("./testdata/variables.pi","rb") as picklefile:
    frag_index,frag_prop,frag_amount,valid_chroms,chroms_offsets = pickle.load(picklefile)

In [4]:
with open("./testdata/peaks_chr1_mumbach.pi" ,"rb") as picklein:
    smoothed_diagonal,refined_peaks = pickle.load(picklein)

In [5]:
CSR_mat_full = scipy.sparse.load_npz('./testdata/sparse_matrix_mumbach_non_reassigned_chr1.npz')

In [18]:
#return list of tuples with start and end indexes for peaks. optionally filter by things larger than something

peaks=[]

i = 0
while i < len(refined_peaks):
    if refined_peaks[i] == 1:
        start = i
        while refined_peaks[i] == 1:
            i=i+1
        end = i
        peaks.append((start,end))
    i=i+1

print(len(peaks))



3158


In [36]:
#interactive visualization of peaks viewpoints
%matplotlib qt
from ipywidgets import *
fig = matplotlib.pyplot.figure()
ax = fig.add_subplot(111)
matplotlib.pyplot.ion()
def interactplot(i):
    ax.clear()
    chip=smoothed_diagonal[peaks[i][0]-1500:peaks[i][1]+1500]
    bait=moving_integration(CSR_mat_full[peaks[i][0]:peaks[i][1],].toarray().sum(axis=0).tolist()[peaks[i][0]-1500:peaks[i][1]+1500],3)
    ax.plot(chip)
    ax.plot(bait)
    fig.canvas.draw()
    
    ratio = []
    for x, y in zip(bait, chip):
        ratio.append(x*100/(y+10))
    ax.plot(moving_integration(ratio,20))
    
    print(frag_prop[peaks[i][0]])
    print(frag_prop[peaks[i][0]-1500])
    print(frag_prop[peaks[i][1]+1500])
interact(interactplot,i=widgets.IntSlider(min=1,max=len(peaks)-1,step=1,value=1));

interactive(children=(IntSlider(value=1, description='i', max=3157, min=1), Output()), _dom_classes=('widget-i…